In [1]:
import base64
import requests
from openai import OpenAI
# OpenAI API Key
api_key = "Insert-Your-OpenAi-API-Key-Here"

# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')
headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
}
client = OpenAI(api_key = api_key)
def question_image(url,query,detail="low"):
    if url.startswith("http://")or url.startswith("https://"):
        response = client.chat.completions.create(
            model="gpt-4-vision-preview",
            messages=[
            {
            "role": "user",
            "content": [
                {"type": "text", "text": f"{query}"},
                    {
                    "type": "image_url",
                    "image_url": url,
                    },
                ],
            }
        ],
        max_tokens=1000,
            
        )
        return response.choices[0].message.content
    else:
        
        base64_image = encode_image(url)

        payload = {
            "model": "gpt-4-vision-preview",
            "messages": [
              {
                "role": "user",
                "content": [
                  {
                    "type": "text",
                    "text": f"{query}?"
                  },
                  {
                    "type": "image_url",
                    "image_url": {
                      "url": f"data:image/jpeg;base64,{base64_image}"
                    },
                  }
                ]
              }
            ],
            "max_tokens": 1000
        }

        response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

        temp=response.json()
        return temp['choices'][0]['message']['content']

In [2]:
query="Extract the airline_name,passenger_name,flight_num,departure_city,destination_city and date_of_departure from this boarding pass in a JSON format"
image_url=r"singapore.jpg"
abc=question_image(image_url,query)
print(abc)

```json
{
  "airline_name": "Singapore Airlines",
  "passenger_name": "Heng Kok Hong Mr",
  "flight_num": "SQ 231",
  "departure_city": "SIN (Singapore)",
  "destination_city": "SYD (Sydney)",
  "date_of_departure": "19JAN18"
}
```
